In [ ]:
import numpy as np

import torch
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream_custom_range import data_stream_custom_range

#from code.source.utilities import data_stream_custom_range

class ShapeSpace(RiemannianManifold):

    def __init__(self, positions, angular_coordinates):
        self.positions = positions
        self.angular_coordinates = angular_coordinates

    def torchComputeAngle(self, poses):
        combos = torch.tensor([[0, 1], [1, 2], [2, 0]])
        ab = torch.norm(poses[combos[0, 0], :] - poses[combos[0, 1], :])
        bc = torch.norm(poses[combos[1, 0], :] - poses[combos[1, 1], :])
        ca = torch.norm(poses[combos[2, 0], :] - poses[combos[2, 1], :])
        output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
        return (output)

    def torchCompute3angles(self, position):
        angles = np.ones(3)
        gradients = np.zeros((3, position.shape[0], 3))
        for i in range(3):
            #print(i)
            poses = torch.tensor(position[[i, (i + 1) % 3, (i + 2) % 3], :], requires_grad=True)
            tempang = self.torchComputeAngle(poses)
            tempang.backward()
            angles[i] = tempang.detach().numpy()
            gradients[i] = poses.grad[[(2 * i) % 3, (2 * i + 1) % 3, (2 * i + 2) % 3], :]
            # del(poses)
        return(angles, gradients)

    def reshapepointdata(self, pointdata, atoms3):
        natoms = len(np.unique(atoms3))
        output = np.zeros((pointdata.shape[0] * pointdata.shape[1], natoms * 3))
        for i in range(pointdata.shape[0]):
            for j in range(pointdata.shape[1]):
                for k in range(pointdata.shape[2]):
                    for l in range(pointdata.shape[3]):
                        # print(atoms3[k]*3 + l)
                        output[i * 3 + j, atoms3[i][k] * 3 + l] = pointdata[i, j, k, l]
        return(output)

    def get_wilson(self, selind, atoms3, tdata):
        natoms = len(np.unique(atoms3))
        jacobien = np.zeros((len(selind), len(atoms3) * 3, natoms * 3))
        for i in range(len(selind)):
            pointdata = tdata[i * len(atoms3): (i + 1) * len(atoms3)]
            jacobien[i] = self.reshapepointdata(pointdata, atoms3)
        return(jacobien)

    def get_internal_projector(self, natoms, jacobien, selind):
        nnonzerosvd = 3 * natoms - 7
        internalprojector = np.zeros((len(selind), jacobien.shape[1], nnonzerosvd))
        for i in range(len(selind)):
            asdf = np.linalg.svd(jacobien[i])
            internalprojector[i] = (asdf[0][:, :nnonzerosvd] )
        return (internalprojector)

    def get_dw(self,cores,atoms3,natoms, selected_points):
        positions = self.positions
        self.selected_points = selected_points
        p = Pool(cores)
        n = len(selected_points)
        results = p.map(lambda i: self.torchCompute3angles(position=positions[i[0], atoms3[i[1]], :]),
                        data_stream_custom_range(selected_points, len(atoms3)))
        tdata = np.asarray([results[i][1] for i in range(n * len(atoms3))])
        # for i in range(len(selected_points)):
        #     pointdata = tdata[i * len(atoms3): (i + 1) * len(atoms3)]
        jacobien = self.get_wilson(selected_points, atoms3, tdata)
        internalprojector = self.get_internal_projector(natoms, jacobien, selected_points)
        return(internalprojector)

    def torchComputeTorsion(self, poses):
        #combos = torch.tensor([[0, 1], [1, 2], [2, 0]])
        #ab = torch.norm(poses[combos[0, 0], :] - poses[combos[0, 1], :])
        #bc = torch.norm(poses[combos[1, 0], :] - poses[combos[1, 1], :])
        #ca = torch.norm(poses[combos[2, 0], :] - poses[combos[2, 1], :])
        #output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
        #return (output)
        
    def positions_to_torsion(self, positions4):
        positions4 = torch.tensor(positions4, requires_grad=True)
        d1 = positions4[0]
        c1 = positions4[1]
        c2 = positions4[2]
        d2 = positions4[3]
        cc = c2 - c1
        ip = torch.sum((d1 - c1) * (c2 - c1)) / (torch.sum((c2 - c1) ** 2))
        tilded1 = [d1[0] - ip * cc[0], d1[1] - ip * cc[1], d1[2] - ip * cc[2]]
        iq = torch.sum((d2 - c2) * (c1 - c2)) / (torch.sum((c1 - c2) ** 2))
        cc2 = (c1 - c2)
        tilded2 = [d2[0] - iq * cc2[0], d2[1] - iq * cc2[1], d2[2] - iq * cc2[2]]
        tilded2star = [tilded2[0] + cc2[0], tilded2[1] + cc2[1], tilded2[2] + cc2[2]]
        ab = torch.sqrt((tilded2star[0] - c1[0]) ** 2 + (tilded2star[1] - c1[1]) ** 2 + (tilded2star[2] - c1[2]) ** 2)
        bc = torch.sqrt((tilded2star[0] - tilded1[0]) ** 2 + (tilded2star[1] - tilded1[1]) ** 2 + (
                    tilded2star[2] - tilded1[2]) ** 2)
        ca = torch.sqrt((tilded1[0] - c1[0]) ** 2 + (tilded1[1] - c1[1]) ** 2 + (tilded1[2] - c1[2]) ** 2)
        output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
        return (output)
    
    def g4(self, positions4, grad=True):
        positions4 = torch.tensor(positions4, requires_grad=True)
        torsion = self.positions_to_torsion(positions4)
        torsion.backward(retain_graph=True)
        return (torsion, positions4.grad)
    
    def get_g(self,x):
        atoms3 = self.atoms3
        atoms4 = self.atoms4
        p = self.p

        output = np.zeros(p)
        combos = np.asarray([[0, 1, 2], [1, 2, 3], [0, 2, 3], [0, 1, 3]])
        for k in range(p):
            atom4 = atoms4[k, :]
            angles4 = []
            # get identities of triangles on boundary of tetrahedron
            actived = np.zeros(4)
            for i in range(4):
                actived[i] = np.where([set(item).issubset(atom4[combos[i, :]]) for item in atoms3])[0][0]
            actived = np.asarray(actived, dtype=int)
            naive = np.reshape(x, (int(x.shape[0] / 3), 3))[actived, :]
            for i in range(4):
                a = atoms3[actived[i]]
                b = atom4[np.in1d(atom4, atoms3[actived[i]])]
                for j in range(3):
                    angles4.append(naive[i, np.where(a == b[j])[0]])
            # the jth position in the ith row contains the gradient corresponding to the jth position in the truncated atom4
            a4 = np.reshape(angles4, (4, 3))
            fitin = self.g4(a4)
            # plus the lowest index first
            output[k] = fitin
        return (output)

In [ ]:
threeangles = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]), data_stream(10000, 84))
torsions = p.map(lambda i: compute3angles(position=positions[i[0], atoms4[i[1]], :]), data_stream(10000, 84))